In [1]:
%matplotlib notebook

import os
import glob
import zipfile
import shutil

import pandas

import gp

In [2]:
source_dir = "/Users/dlahr/temp/H202SC20040591_dge/analysis/"
experiment_id = "H202SC20040591"

dge_stats_for_rnk_list = ["logFC", "t"]

num_permutations = 1000

job_memory = "4 Gb"

# guidance from GenePattern forum:
# gseapreranked_job_spec.set_parameter("job.memory", "2 Gb")
# gseapreranked_job_spec.set_parameter("job.queue", "gp-cloud-default")
# gseapreranked_job_spec.set_parameter("job.cpuCount", "1")
# gseapreranked_job_spec.set_parameter("job.walltime", "02:00:00")

# notes from running if any
Job 257826 failed for some reason, restarted manually on server, will download / unzip manually as well when ready

# prepare directories for output

In [3]:
gsea_dir = os.path.join(source_dir, "gsea")
print("gsea_dir:", gsea_dir)
if os.path.exists(gsea_dir):
    shutil.rmtree(gsea_dir)
os.mkdir(gsea_dir)

rnk_dir = os.path.join(gsea_dir, "rnk")
print("rnk_dir:", rnk_dir)
os.mkdir(rnk_dir)

gsea_dir: /Users/dlahr/temp/H202SC20040591_dge/analysis/gsea
rnk_dir: /Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk


# build rnk files from DGE data files

In [4]:
dge_file_list = glob.glob(
    os.path.join(source_dir, "dge_data", experiment_id + "_*_DGE_r*.txt")
)
dge_file_list.sort()
print(len(dge_file_list))
print("\n".join(dge_file_list))

4
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_DGE_r30500x10.txt
/Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_DGE_r30500x10.txt


In [5]:
input_rnk_files_list = []

for dge_file in dge_file_list[:]:
    print("dge_file:", dge_file)
    dge_df = pandas.read_csv(dge_file, sep="\t")
    print(dge_df.shape)
    
    base_dge_filename = os.path.splitext(os.path.basename(dge_file))[0]
    print(base_dge_filename)
    
    base_output_filename = "_".join(base_dge_filename.split("_")[:-2])
    print("base_output_filename:", base_output_filename)
    
    for dge_stat_for_rnk in dge_stats_for_rnk_list:
        rnk_df = dge_df.loc[~pandas.isnull(dge_df.gene_symbol), 
                            ["gene_symbol", dge_stat_for_rnk]].copy()
        new_cols = list(rnk_df.columns)
        new_cols[0] = "#" + new_cols[0]
        rnk_df.columns = new_cols
        display(rnk_df.head())
        
        output_rnk_file = base_output_filename + "_{dge_stat}_r{rows}x{cols}.rnk".format(
            dge_stat=dge_stat_for_rnk, rows=rnk_df.shape[0], cols=rnk_df.shape[1]
        )
        print(output_rnk_file)
        
        output_filepath = os.path.join(rnk_dir, output_rnk_file)
        input_rnk_files_list.append(output_filepath)
        rnk_df.to_csv(output_filepath, sep="\t", index=False)
    
    print()

print(len(input_rnk_files_list))
print("\n".join(input_rnk_files_list))

dge_file: /Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_DGE_r30500x10.txt
(30500, 11)
H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_DGE_r30500x10
base_output_filename: H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h


,#gene_symbol,logFC
0,RP11-160E2.16,-3.495766
1,RP1-99E18.2,-3.032918
2,RP11-386M24.9,-2.944619
3,RP11-468E2.10,-2.915015
4,TMEFF2,-2.797327


H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.rnk


,#gene_symbol,t
0,RP11-160E2.16,-4.864835
1,RP1-99E18.2,-4.682609
2,RP11-386M24.9,-3.918192
3,RP11-468E2.10,-3.826496
4,TMEFF2,-3.284992


H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.rnk

dge_file: /Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_DGE_r30500x10.txt
(30500, 11)
H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_DGE_r30500x10
base_output_filename: H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h


...

H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_t_r30144x2.rnk

dge_file: /Users/dlahr/temp/H202SC20040591_dge/analysis/dge_data/H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_DGE_r30500x10.txt
(30500, 11)
H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_DGE_r30500x10
base_output_filename: H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h


,#gene_symbol,logFC
0,RP11-160E2.17,-5.171117
1,RP11-160E2.16,-5.075931
2,COL10A1,-4.861402
3,AMHR2,-4.534430
4,AC007952.4,-4.166896


H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_logFC_r30144x2.rnk


,#gene_symbol,t
0,RP11-160E2.17,-9.588262
1,RP11-160E2.16,-8.472290
2,COL10A1,-6.522934
3,AMHR2,-8.484385
4,AC007952.4,-7.973930


H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_t_r30144x2.rnk

8
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_logFC_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_t_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_logFC_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_t_r30144x2.rnk
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/rnk/H202SC20040591_FHT3794_MP46_QDx6_24h_Vehicle_MP46_QDx6_24h_logFC_r30144x2.rnk
/Users/dlahr

# run GSEA using rnk files

In [6]:
reference_genesets = [
    ("all", {"c1.all.v7.1.symbols.gmt [Positional]", "c2.all.v7.1.symbols.gmt [Curated]", "c3.all.v7.1.symbols.gmt [Motif]",
     "c5.all.v7.1.symbols.gmt [Gene Ontology]", "c6.all.v7.1.symbols.gmt [Oncogenic Signatures]",
     "c7.all.v7.1.symbols.gmt [Immunologic signatures]", "h.all.v7.1.symbols.gmt [Hallmarks]"}),
    ("just_hallmarks", {"h.all.v7.1.symbols.gmt [Hallmarks]"})
]
print(len(reference_genesets))
print([(k,len(v)) for k,v in reference_genesets])

2
[('all', 7), ('just_hallmarks', 1)]


In [7]:
gp_username = "dllahr"
gp_password = "yQJ7a6m^mBE07#845@Ll#AJq!"
gp_url = "https://cloud.genepattern.org/gp"

In [8]:
# Create a GenePattern server proxy instance
gpserver = gp.GPServer(gp_url, gp_username, gp_password)

In [9]:
# upload input files
input_gp_files_list = []

for input_file in input_rnk_files_list:
    uploaded_file_name = os.path.basename(input_file)
    print(uploaded_file_name)

    input_gp_file = gpserver.upload_file(uploaded_file_name, input_file)
#     print(dir(input_gp_file))
    print(input_gp_file.get_url())
    input_gp_files_list.append(input_gp_file)

H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.rnk
https://cloud.genepattern.org/gp/users/dllahr/tmp/run5687728369687963663.tmp/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.rnk
H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.rnk
https://cloud.genepattern.org/gp/users/dllahr/tmp/run7751554138625777331.tmp/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.rnk
H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_logFC_r30144x2.rnk
https://cloud.genepattern.org/gp/users/dllahr/tmp/run1005952227850521048.tmp/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_logFC_r30144x2.rnk
H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_t_r30144x2.rnk
https://cloud.genepattern.org/gp/users/dllahr/tmp/run7904784910493905373.tmp/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_t_r30144x2.rnk
H202SC20040591_FHT3794_MP46_QDx1_24h_Vehicle_MP46_QDx1_24h_logFC_r30144x2.rnk
https://cloud.genepatt

In [10]:
# Get the list of tasks
task_list = gpserver.get_task_list()

In [11]:
# print("\n".join([x.get_name() for x in task_list]))

In [12]:
gsea_task_list = [x for x in task_list if "gsea" in x.get_name().lower()]
print(len(gsea_task_list))
print("\n".join([x.get_name() for x in gsea_task_list]))

4
GSEA
GSEALeadingEdgeViewer
GSEAPreranked
ssGSEA


In [13]:
t = [x for x in gsea_task_list if x.get_name() == "GSEAPreranked"]
assert len(t) == 1
gsea_preranked_task = t[0]
print(dir(gsea_preranked_task))
print(gsea_preranked_task.name)
print(gsea_preranked_task.lsid)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_params_loaded', 'description', 'documentation', 'dto', 'get_description', 'get_lsid', 'get_name', 'get_parameters', 'get_version', 'job', 'job_number', 'job_spec', 'json', 'lsid', 'make_job_spec', 'name', 'param_load', 'params', 'server_data', 'submit_json', 'uri', 'version']
GSEAPreranked
urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00317:7.0.4


In [14]:
gsea_preranked_module = gp.GPTask(gpserver, gsea_preranked_task.lsid)
print(dir(gsea_preranked_module))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_params_loaded', 'description', 'documentation', 'dto', 'get_description', 'get_lsid', 'get_name', 'get_parameters', 'get_version', 'job', 'job_number', 'job_spec', 'json', 'lsid', 'make_job_spec', 'name', 'param_load', 'params', 'server_data', 'submit_json', 'uri', 'version']


In [15]:
gsea_preranked_module.param_load()

In [16]:
# Get the list of GPTaskParam objects
params_list = gsea_preranked_module.get_parameters()

for param in params_list:              # Loop through each parameter
    print( param.get_name() )          # Print the parameter's name
    print( "type:", param.get_type() )          # Print the parameter's type (text, number, file, etc.)
    print( param.get_description() )   # Print the parameter's description
    print( "default_value:", param.get_default_value() ) # Print the parameter's default value
    print( "is_optional:", param.is_optional() )       # Print whether the parameter is optional
    print( '' )                        # Leave a blank line between printed parameters

ranked.list
type: File
Rank ordered gene list - .rnk
default_value: None
is_optional: False

gene.sets.database
type: File
Gene sets database from GSEA website.
default_value: None
is_optional: False

number.of.permutations
type: String
Number of permutations to perform
default_value: 1000
is_optional: False

collapse.dataset
type: String
Select whether to collapse each probe set in the expression dataset into a single vector for the gene, which gets identified by its gene symbol.  It is also possible to remap symbols from one namespace to another without collapsing (an error will occur if multiple source genes map to a single destination gene).  No_Collapse will use the dataset as-is.
default_value: Remap_Only
is_optional: False

chip.platform.file
type: File
DNA Chip (array) annotation file from GSEA website.  Upload your own chip file if the one corresponding to your DNA Microarray platform is not listed in the drop-down menu.  A chip file is only required if collapse dataset is set

In [17]:
# Loop through each parameter
for param in params_list:
    if param.is_choice_param():        # If the parameter is a choice param
        print( param.get_name() )      # Print the parameter's name
        
        choices = param.get_choices()  # Get a list of valid choices 
        for choice in choices:         # Print the label and value for each choice
            print( choice['label'] + " = " + choice['value'] )
            
        # Print the default selected value for each choice
        print( param.get_choice_selected_value() )
        print()

gene.sets.database
c1.all.v7.1.symbols.gmt [Positional] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c1.all.v7.1.symbols.gmt
c2.all.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.all.v7.1.symbols.gmt
c2.cgp.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cgp.v7.1.symbols.gmt
c2.cp.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cp.v7.1.symbols.gmt
c2.cp.biocarta.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cp.biocarta.v7.1.symbols.gmt
c2.cp.kegg.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cp.kegg.v7.1.symbols.gmt
c2.cp.pid.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.cp.pid.v7.1.symbols.gmt
c2.cp.reactome.v7.1.symbols.gmt [Curated] = ftp://gpftp.broadinstitute.org/module_support

In [18]:
genesets_param = [x for x in params_list if x.get_name().startswith("gene.sets.database")]
assert len(genesets_param) == 1, len(genesets_param)
genesets_param = genesets_param[0]

reference_geneset_urls = []
for group_name, group_geneset_labels in reference_genesets:
    print(group_name)
    
    matching_geneset_choices = [x for x in genesets_param.get_choices() if x["label"] in group_geneset_labels]
    print(len(matching_geneset_choices))
    check = set([x["label"] for x in matching_geneset_choices])
    assert check == group_geneset_labels, (len(check), len(group_geneset_labels), check, group_geneset_labels)
    
    reference_geneset_urls.append((
        group_name, [x["value"] for x in matching_geneset_choices]
    ))

print()
print(len(reference_geneset_urls))
print("\n\n".join([str(x) for x in reference_geneset_urls]))

all
7
just_hallmarks
1

2
('all', ['ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c1.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c2.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c3.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c5.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c6.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c7.all.v7.1.symbols.gmt', 'ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/h.all.v7.1.symbols.gmt'])

('just_hallmarks', ['ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/h.all.v7.1.symbols.gmt'])


In [19]:
all_job_spec_list = []

change_default_params = {"number.of.permutations": str(num_permutations),
                        "collapse.dataset": "No_Collapse"}

add_params = {"job.memory":job_memory}
    
for input_gp_file in input_gp_files_list:
    for gs_group_name, gs_urls in reference_geneset_urls:

        # Create the GPJobSpec
        job_spec = gsea_preranked_module.make_job_spec()
#         print(dir(job_spec))

        for param_name,param_value in add_params.items():
            job_spec.set_parameter(param_name, param_value)

        # Loop through all the parameters and set their default values
        for param in gsea_preranked_module.get_parameters():  
            # If the parameter has a default value, set that value
            param_name = param.get_name()
            param_default_value = param.get_default_value()
#             print(param_name, param_default_value)

            if param_name in change_default_params:
                job_spec.set_parameter(param_name, change_default_params[param_name])
            elif param.get_default_value() != None: 
                # Set the default value
                job_spec.set_parameter(param_name, param_default_value)

        # Attach the input file to the correct parameter
        job_spec.set_parameter("ranked.list", input_gp_file.get_url()) 

        job_spec.set_parameter("gene.sets.database", gs_urls)
            #["ftp://gpftp.broadinstitute.org/module_support_files/msigdb/gmt/c1.all.v7.1.symbols.gmt"])

        # job_spec.set_parameter("number.of.permutations", "10")
#         job_spec.params

        all_job_spec_list.append((gs_group_name, job_spec))

print(len(all_job_spec_list))
print([x for x in all_job_spec_list])

16
[('all', <gp.core.GPJobSpec object at 0x7fcbf2f82410>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf2f82390>), ('all', <gp.core.GPJobSpec object at 0x7fcbf2f823d0>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf2f822d0>), ('all', <gp.core.GPJobSpec object at 0x7fcbf2f82310>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf2f82290>), ('all', <gp.core.GPJobSpec object at 0x7fcbf2f82190>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf2f82790>), ('all', <gp.core.GPJobSpec object at 0x7fcbf2f82810>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf2f82250>), ('all', <gp.core.GPJobSpec object at 0x7fcbf4551210>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf4551290>), ('all', <gp.core.GPJobSpec object at 0x7fcbf4551410>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf4551350>), ('all', <gp.core.GPJobSpec object at 0x7fcbf4551550>), ('just_hallmarks', <gp.core.GPJobSpec object at 0x7fcbf4551450>)]


In [20]:
print(all_job_spec_list[0][0])
t = all_job_spec_list[0][1]
dir(t)
t.params

all


[{'name': 'job.memory', 'values': ['4 Gb']},
 {'name': 'number.of.permutations', 'values': ['1000']},
 {'name': 'collapse.dataset', 'values': ['No_Collapse']},
 {'name': 'scoring.scheme', 'values': ['weighted']},
 {'name': 'max.gene.set.size', 'values': ['500']},
 {'name': 'min.gene.set.size', 'values': ['15']},
 {'name': 'collapsing.mode.for.probe.sets.with.more.than.one.match',
  'values': ['Max_probe']},
 {'name': 'normalization.mode', 'values': ['meandiv']},
 {'name': 'omit.features.with.no.symbol.match', 'values': ['true']},
 {'name': 'make.detailed.gene.set.report', 'values': ['true']},
 {'name': 'num.top.sets', 'values': ['20']},
 {'name': 'random.seed', 'values': ['timestamp']},
 {'name': 'create.svgs', 'values': ['false']},
 {'name': 'output.file.name', 'values': ['<ranked.list_basename>.zip']},
 {'name': 'create.zip', 'values': ['true']},
 {'name': 'dev.mode', 'values': ['false']},
 {'name': 'ranked.list',
  'values': ['https://cloud.genepattern.org/gp/users/dllahr/tmp/run568

In [21]:
job_list = []

for gs_group_name, job_spec in all_job_spec_list:
    # This will return the job object and continue execution even if the job isn't finished
    job = gpserver.run_job(job_spec, False)
#     print(dir(job))

    # Prints a brief description of the job's current state
    print( job.get_status_message() )

    # Quaries the server and returns True if the job is complete, False otherwise
    print( job.is_finished() )

    job_list.append((gs_group_name, job))
#     job.wait_until_done()

for i, (gs_group_name, job) in enumerate(job_list):
    print("waiting on job index i:  ", i)
    job.wait_until_done()

Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has not been submitted to an external queuing system
False
Pending in the GenePattern queue, it has

In [22]:
zip_dir = os.path.join(gsea_dir, "zip_files")
if not os.path.exists(zip_dir):
    os.mkdir(zip_dir)

In [23]:
zip_files_list = []

no_zip_files = []

for gs_group_name, job in job_list:
#     print(gs_group_name)

    t = [x for x in job.get_output_files() if x.get_name().endswith(".zip")]
    
    if len(t) == 1:
        zip_output_file = t[0]
    #     print(zip_output_file.get_name())
    #     print(zip_output_file.get_url())

        dl_filename = os.path.splitext(zip_output_file.get_name())[0] + "_" + gs_group_name + ".zip"
    #     print(dl_filename)

        dl_filepath = os.path.join(zip_dir, dl_filename)
        zip_files_list.append(dl_filepath)

        print(zip_output_file.get_url())
        print(dl_filepath)

        f = open(dl_filepath, "wb")
        f.write(zip_output_file.open().read())
        f.close()
    elif len(t) == 0:
        print("********************* ALERT! ************************")
        print("Zip file not found may mean job failed to run")
        print("gs_group_name:", gs_group_name)
        no_zip_files.append((gs_group_name, job))
        print("*****************************************************")
    else:
        print("********************* WARNING ************************")
        print("More than 1 zip file found in job output, skipping")
        print("gs_group_name:", gs_group_name)
        print("*****************************************************")
    
print()
print("#####################################")
print("info about zip_files_list:")
print(len(zip_files_list))
print("\n".join(zip_files_list))
print()


https://cloud.genepattern.org/gp/jobResults/257812/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.zip
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/zip_files/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2_all.zip
https://cloud.genepattern.org/gp/jobResults/257813/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2.zip
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/zip_files/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2_just_hallmarks.zip
https://cloud.genepattern.org/gp/jobResults/257814/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.zip
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/zip_files/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2_all.zip
https://cloud.genepattern.org/gp/jobResults/257815/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2.zip
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/zip_files/H202SC200

In [24]:
for gs_group_name, job in no_zip_files:
    print(gs_group_name)
    print(job.job_number)
    print(job.get_job_status_url())
    print([x.get_name() for x in job.get_output_files()])

all
257826
https://cloud.genepattern.org/gp/pages/index.jsf?jobid=257826
['stderr.txt', 'stdout.txt']


In [25]:
# dir(no_zip_files[0][1])
# no_zip_files[0][1].has

In [26]:
# t = no_zip_files[0][1]
# t.get_output_files()[0].get_name()

In [27]:
for zip_file in zip_files_list:
#     print(zip_file)
    
    dir_name = os.path.splitext(
        os.path.basename(zip_file)
    )[0]
    
    extract_dir = os.path.join(gsea_dir, dir_name)
    print(extract_dir)
    
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    os.mkdir(extract_dir)
    
    my_zipfile = zipfile.ZipFile(zip_file, "r")
    my_zipfile.extractall(extract_dir)


/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2_all
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_logFC_r30144x2_just_hallmarks
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2_all
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx1_24h_Vehicle_921_QDx1_24h_t_r30144x2_just_hallmarks
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_logFC_r30144x2_all
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_logFC_r30144x2_just_hallmarks
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx6_24h_Vehicle_921_QDx6_24h_t_r30144x2_all
/Users/dlahr/temp/H202SC20040591_dge/analysis/gsea/H202SC20040591_FHT3794_921_QDx6_24h_Veh